In [1]:
:opt no-lint
{-# LANGUAGE TypeSynonymInstances FlexibleInstances #-}

이제 그림 \ref{fig:bigStepCBV}와 \ref{fig:bigStepCBN}의 큰걸음 의미구조를 하스켈로 옮겨 보자.

In [2]:
type Nm = String -- 변수 이름은 문자열로
data Expr = Var Nm         -- x
          | Lam Nm Expr    -- (λx.e)
          | App Expr Expr  -- (e1 e2)
          deriving (Eq, Ord)

In [27]:
instance Show Expr where -- LaTeX 소스코드 생성하는 Show 인스턴스 직접 선언
  showsPrec _ (Var x) = showString x
  showsPrec p (Lam x e) = showParen (p > 1) $
      showString ("\\lambda "++x++".") . showsPrec 1 e
  showsPrec p (App e1 e2) = showParen (p > 9) $
      showsPrec 9 e1 . showString "\\;" . showsPrec 10 e2

import IHaskell.Display (latex)
instance IHaskellDisplay Expr where -- 노트북에 디스플레이하는 인스턴스 선언
    display e = display [latex $ "$"++show e++"$"]

In [4]:
Lam "x" (Lam "y" (App (Var "x") (Lam "z"  (Var "y"))))

In [5]:
-- 연산자 (.), ($) 및 역따옴표 중위 표현 활용 
Lam "x" . Lam "y" $ Var "x" `App` Lam "z" (Var "y")

In [6]:
type Env = [(Nm,Clos)] -- 값계산 환경
data Clos = Cl Expr Env deriving (Eq,Ord,Show) -- 클로저

In [7]:
import Data.List (intercalate)
instance {-# OVERLAPS #-} Show Env where
    show env = "\\{"++ intercalate ",\\," (map show env) ++"\\}"
instance {-# OVERLAPS #-} Show (Nm,Clos) where
    show (x,cl) = "\\langle "++x++"{\\mapsto}"++show cl++"\\rangle "
instance Show Clos where
    show (Cl e env) = "\\langle "++show e++","++show env++"\\rangle "

instance {-# OVERLAPS #-} IHaskellDisplay Env where
    display e = display [latex $ "$"++show e++"$"]
instance {-# OVERLAPS #-} IHaskellDisplay (Nm,Clos) where
    display e = display [latex $ "$"++show e++"$"]
instance IHaskellDisplay Clos where
    display e = display [latex $ "$"++show e++"$"]

In [8]:
env01 = [ ("f", Cl (Lam "z" $ Var "z") []) ]

In [9]:
env01

In [10]:
env02 = [ ( "x", Cl (Lam "z" $ Var "z")               []   ),
          ( "y", Cl (Lam "x" $ Var "f" `App` Var "x") env01) ]

In [11]:
env02

In [12]:
mapM display it -- 직전 실행한 값계산 환경의 각 대응 항목을 한줄씩 디스플레이

In [13]:
:type lookup

lookup :: forall a b. Eq a => a -> [(a, b)] -> Maybe b

In [14]:
lookup "y" env02 -- Maybe Env 디스플레이 인스턴스를 선언하지 않아 텍스트 형태로 출력
lookup "z" env02 -- 환경에 없는 변수 참조

Just \langle \lambda x.f\;x,\{\langle f{\mapsto}\langle \lambda z.z,\{\}\rangle \rangle \}\rangle

Nothing

In [15]:
lookupEnv :: Nm -> Env -> [Clos]
lookupEnv x env = case lookup x env of Nothing -> []
                                       Just cl -> [cl]

In [16]:
lookupEnv "y" env02 -- 리스트 원소 한줄씩 display되는데 한개이므로

In [17]:
lookupEnv "z" env02 -- 빈 리스트라 아무것도 display되지 않음

In [18]:
bigStepCBV :: Clos -> [Clos]
bigStepCBV (Cl (Var x)   env) = lookupEnv x env
bigStepCBV (Cl (Lam x e) env) = undefined -- 연습문제로 작성
bigStepCBV (Cl (App x e) env) = undefined -- 연습문제로 작성

In [19]:
bigStepCBV (Cl (Var "y") env02) -- 큰걸음 실행 결과 있음 (환경에 있는 변수)

In [20]:
bigStepCBV (Cl (Var "z") env02) -- 큰걸음 실행 결과 없음 (환경에 없는 변수)

In [21]:
-- bigStepCBV (Cl (Lam ...) ...) -- 연습문제로 작성한 코드 관련 테스트 작성

In [22]:
-- bigStepCBV (Cl (App ...) ...) -- 연습문제로 작성한 코드 관련 테스트 작성

여기까지 인자 먼저 값계산의 큰걸음 의미구조(그림 \ref{fig:bigStepCBV})를 하스켈로 옮긴 함수 `bigStepCBV`를 작성하여 테스트하는 보았다면,
또 하나의 연습문제로 적용 먼저 값계산의 큰걸음 의미구조(그림 \ref{fig:bigStepCBN})를 하스켈로 옮긴 함수 `bigStepCBN`도 마찬가지 방법으로
작성하고 테스트해 볼 수 있을 것이다. 같은 클로저를 `bigStepCBN`으로 실행하면 정상적으로 실행되어 결과값이 있는데
`bigStepCBV`는 정상적으로 실행되지 않고 결과값이 없는 경우가 없을지 생각해 보고 있다면 그러한 예시를 구성해 테스트해 보라.
또한 같은 클로저를 `bigStepCBN`과 `bigStepCBV`로 실행했을 때 모두 정상 실행되지만 두 결과값이 똑같지는 않은 경우는 없는지
생각해 보고 있다면 그러한 예시를 구성해 테스트해 보라.

In [23]:
bigStepCBN :: Clos -> [Clos]
bigStepCBN (Cl (Var x)   env) = undefined -- 연습문제로 작성
bigStepCBN (Cl (Lam x e) env) = undefined -- 연습문제로 작성
bigStepCBN (Cl (App x e) env) = undefined -- 연습문제로 작성

In [24]:
-- 연습문제로 작성한 bigStepCBN에 대한 테스트

# 값계산 환경을 활용한 작은걸음 의미구조
\label{sec:evalCEK}
이 장을 시작하며 \ref{sec:evalSmallStep}절 앞부분에서는 줄임(reduction)과 값계산(evaluation)을 연관지어 이해하기 좋도록
타입없는 람다계산법의 값계산을 맥락적 의미구조로 설명하였다. 그런데, 이런 기반의 의미구조를 그대로 구현하한다면 불필요한 치환이
발생하는 단점을 있다. 이런 단점이 해소된 값계산 환경을 활용하는 방식의 큰걸음 의미구조를 \ref{sec:evalBigStep}절에서 살펴보았다.
환경을 활용하는 큰걸음 의미구조가 있다면 이에 더 직접적으로 대응되는 값계산 환경을 활용한 작은걸음 의미구조도 생각해 볼 수 있다.
이러한 작은걸음 의미구조가 바로 \`CEK기계'\cite{Felleisen86a}다.\footnote{여기서는
    조금 단순화된 형태의 다음과 같은 하스켈 코드를 참고해 정리하였다.\\$\phantom{AAx}$
    \url{https://matt.might.net/articles/cek-machines/} }

## 인자 먼저 값계산 CEK기계

\begin{align}
\langle x,\sigma\rangle;\kappa
        &\longmapsto
\sigma(x);\kappa
\\
\langle e_1~e_2,\sigma\rangle;\kappa
        &\longmapsto
\langle e_1,\sigma\rangle;\langle \bullet~e_2,\sigma\rangle,\kappa
\\
\langle \lambda x.e,\sigma_1\rangle;\langle \bullet~e_2,\sigma\rangle,\kappa
        &\longmapsto
\langle e_2,\sigma\rangle;\langle(\lambda x.e)\,\bullet,\sigma_1\rangle,\kappa
\\
v_2;\langle(\lambda x.e)\,\bullet,\sigma_1\rangle,\kappa
        &\longmapsto
\langle e,\{x{\mapsto}v_2\}\sigma_1\rangle;\kappa
\end{align}

## 적용 먼저 값계산 CEK기계

\begin{align}
\langle x,\sigma\rangle;\kappa
        &\longmapsto
\sigma(x);\kappa
\\
\langle e_1~e_2,\sigma\rangle;\kappa
        &\longmapsto
\langle e_1,\sigma\rangle;\langle\bullet~e_2,\sigma\rangle,\kappa
\\
\langle \lambda x.e,\sigma_1\rangle;\langle\bullet~e_2,\sigma\rangle,\kappa
        &\longmapsto
\langle e,\{x{\mapsto}\langle e_2,\sigma\rangle\}\sigma_1\rangle;\kappa
\end{align}

\section*{요점정리}
* TODO
* TODO
* TODO

\section*{연습문제}
1. TODO
1. TODO
1. TODO

\section*{탐구과제}
1. TODO
1. TODO
1. TODO